In [ ]:
# Install the transformers package from Hugging Face:
!pip install transformers
!pip install torch
!pip install tensorflow

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces


In [ ]:
def recursive_summarize(text, max_length=200, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [ ]:
text = '''Le data mesh est une architecture de données décentralisée qui organise les données par domaine d'activité spécifique (par exemple, le marketing, les ventes, le service client, etc.). Un data mesh permet de transférer la propriété des données aux producteurs d'un jeu de données donné.
Les producteurs de données sont capables de définir des politiques de gouvernance des données axées sur la documentation, la qualité et l'accès. Bien que cette approche fédérée élimine de nombreux goulots d'étranglement opérationnels associés aux systèmes monolithiques centralisés, cela ne signifie pas nécessairement que vous ne pouvez pas utiliser les systèmes de stockage traditionnels, comme les datalakes ou les entrepôts de données. Cela signifie simplement que leur utilisation est passée d'une plateforme de données unique et centralisée à plusieurs référentiels de données décentralisés.
Il convient de noter que le data mesh favorise l'adoption de technologies cloud natives et de plateformes cloud pour évoluer et atteindre les objectifs en termes de gestion des données. Ce concept est couramment comparé aux microservices pour aider le public à comprendre son utilisation dans ce contexte. Cette architecture distribuée étant particulièrement utile pour faire évoluer les besoins en données dans une organisation, on peut en déduire qu'un data mesh peut ne pas convenir à tous les types d'entreprises ; c'est-à-dire que les petites entreprises peuvent ne pas tirer parti d'un data mesh, car leurs données d'entreprise peuvent ne pas être aussi complexes que celles d'une grande organisation.
'''
final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  3
****************************************************
Piece: 1  out of  3 pieces
Le data mesh est une architecture de données décentralisée qui organise les données par domaine d'activité spécifique (par exemple, le marketing, les ventes, le service client, etc.). Un data mesh permet de transférer la propriété des données aux producteurs d'un jeu de données donné. 
Les producteurs de données sont capables de définir des politiques de gouvernance des données axées sur la documentation, la qualité et l'accès. Bien que cette approche fédérée élimine de nombreux goulots d'étranglement opérationnels associés aux systèmes monolithiques centralisés, cela ne signifie pas nécessairement que vous ne pou 

SUMNMARY:  Un data mesh permet de transférer la propriété des données aux producteurs d'un jeu. Cette approche fédérée élimine nombreux goulots d'étranglement opérationnels.
****************************************************
*****